<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Classification/CNN_Classification_automatique-ESSAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CREATION DU MODELE

In [1]:
#-----Imports et installations-----
#Pour se connecter au drive et acceder a ses fichiers
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Pour l'apprentissage profond
import os
from fastai.vision.all import *
from fastai.vision import *
import pandas as pd

Mounted at /content/drive


In [2]:
Espece_visee = "Solcan"
Chemin_donnees = "/content/drive/MyDrive/Projet_mellifere/Asclepias_syriaca"

In [4]:
Noms_classes = []
for Noms_dossiers in os.listdir(Chemin_donnees):
    if ("." in Noms_dossiers) or (Noms_dossiers=="models"): #Un dossier "Models" est cree automatiquement lors de l'entrainement, donc je souhaite l'exclure ici. Aussi, le "." exclut tous les fichiers avec extension
        continue
    else:
        Noms_classes.append(Noms_dossiers)
Noms_classes

['Rejete', 'Trop proche', 'Feuillu', 'Fleuri']

In [7]:
Chemins_images = []
Etiquettes = []

for i in range(len(Noms_classes)):
    Classe_nom = Noms_classes[i]
    if ("." in Classe_nom) or (Classe_nom=="models"):
       continue
    else:
        Noms_images = os.listdir(f"{Chemin_donnees}/{Classe_nom}")
        for Image_nom in Noms_images:
            Image_chemin = f"{Chemin_donnees}/{Classe_nom}/{Image_nom}"
            Chemins_images.append(Image_chemin)
            Etiquettes.append(i)

In [ ]:
df = pd.DataFrame(
    {
        "Nom": Chemins_images,
        "Etiquette": Etiquettes
    }
)
df

In [10]:
df.to_csv(
    path_or_buf=f"{Chemin_donnees}/etiquettes.csv",
    index=False
)

Telechargement des donnees

In [11]:
dls = ImageDataLoaders.from_folder(
      path=Chemin_donnees,
      item_tfms=Resize(224),
      bs=16,
      batch_tfms=[Normalize.from_stats(*imagenet_stats), RandTransform()],
      valid_pct=0.2
)

Creation du modele

In [12]:
Modele = vision_learner(
    dls=dls,
    arch=models.resnet50,
    metrics=[accuracy, error_rate]
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 189MB/s]


In [14]:
Modele.fine_tune(
    epochs=1
)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.571750,0.433420,0.902011,0.097989,1:07:57


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.358692,0.299233,0.908716,0.091284,1:33:29


In [ ]:
# #Nb_epochs = 1
# #Performance = []

# #for epoch in range(Nb_epochs):
#     Modele.fit_one_cycle(1)
#     Modele.save(f"CNN_{Espece_visee}_epoch{epoch}.pth")
#     Perfo = Modele.recorder.final_record
#     Performance.append(Perfo)

VALIDATION ASSISTEE

In [15]:
import shutil
from IPython.display import display
#Pour vider la console (utile pour la boucle)
from IPython.display import clear_output
#Pour creer le widget qui permet l'affichage d'images et de boutons
import ipywidgets as widgets

clear_output()

In [ ]:
#-----Objets a modifier avant le debut du classement-----
#Copier ici le chemin d'acces vers le dossier source (ou se trouvent les fichiers a transferer)
Directory = "/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis"

#-----Objets importants-----
Fichiers = os.listdir(Directory) # Genere une liste des fichiers/dossiers dans le dossier source
index = 0

#-----Definition des fonctions
def Fenetre_choix():
    global index
    global File
    global Chemin_fichier
    if index <= (len(Fichiers)-1):
        File = Fichiers[index]
        if File.endswith(".jpeg"):
            Chemin_fichier = os.path.join(Directory, File) #Cree le chemin d'acces pour le fichier File
            Image_widget = widgets.Image(value=open(Chemin_fichier, "rb").read(), format="jpg", width=300, height=300) #Cree le widget permettant l'affichage de l'image File
            Prediction =
            Bouton_fleuri = widgets.Button(description="Fleuri") #Cree le bouton a cliquer pour l'option "Fleuri"
            Bouton_fleuri.on_click(lambda x: transf_Fleuri()) #Si on clique sur le bouton Fleuri, une fonction temporaire (lambda x) fait le lien entre le clic et la fonction de transfert vers Fleuri
            Bouton_feuillu = widgets.Button(description="Feuillu")
            Bouton_feuillu.on_click(lambda x: transf_Feuillu())
            Bouton_rejete = widgets.Button(description="Rejete")
            Bouton_rejete.on_click(lambda x: transf_Rejete())
            Bouton_proche = widgets.Button(description="Trop proche")
            Bouton_proche.on_click(lambda x: transf_Proche())
            container_widget = widgets.VBox([Image_widget, Bouton_fleuri, Bouton_feuillu, Bouton_proche, Bouton_rejete]) #Ramasse tous les widgets precedemment crees dans une seule fenetre
            display(container_widget)
        else:
            Passer()
    else:
        print("Mission accomplie, 007. Felicitations.")

def transf_Fleuri(): #Definit la fonction qui sera utilisee si l'image est categorisee comme fleurie
    global Chemin_fichier
    Chemin_Fleuri = os.path.join(Directory, "Fleuri")
    os.makedirs(Chemin_Fleuri, exist_ok=True) #cree le dossier destination "Fleuri" dans le dossier source. exist_ok=True fait en sorte que s'il existe deja, le code se poursuit quand meme
    shutil.move(Chemin_fichier, Chemin_Fleuri)
    clear_output()
    global File
    print(f"Image precedente ({File}) transferee dans {Chemin_Fleuri}")
    global index
    index = index+1
    Fenetre_choix()

def transf_Feuillu(): #Definit la fonction qui sera utilisee si l'image est categorisee comme feuillue
    global Chemin_fichier
    Chemin_Feuillu = os.path.join(Directory, "Feuillu")
    os.makedirs(Chemin_Feuillu, exist_ok=True)
    shutil.move(Chemin_fichier, Chemin_Feuillu)
    clear_output()
    global File
    print(f"Image precedente ({File}) transferee dans {Chemin_Feuillu}")
    global index
    index = index+1
    Fenetre_choix()

def transf_Rejete(): #Definit la fonction qui sera utilisee si l'image est rejetee
    global Chemin_fichier
    Chemin_Rejete = os.path.join(Directory, "Rejete")
    os.makedirs(Chemin_Rejete, exist_ok=True)
    shutil.move(Chemin_fichier, Chemin_Rejete)
    clear_output()
    global File
    print(f"Image precedente ({File}) transferee dans {Chemin_Rejete}")
    global index
    index = index+1
    Fenetre_choix()

def transf_Proche(): #Definit la fonction qui sera utilisee si l'image est categorisee comme trop proche
    global Chemin_fichier
    Chemin_Proche = os.path.join(Directory, "Trop proche")
    os.makedirs(Chemin_Proche, exist_ok=True)
    shutil.move(Chemin_fichier, Chemin_Proche)
    clear_output()
    global File
    print(f"Image precedente ({File}) transferee dans {Chemin_Proche}")
    global index
    index = index+1
    Fenetre_choix()

def Passer(): #Definit ce qui se passe lorsqu'un fichier n'est pas en .jpeg
    print(f"fichier -{File}- ignore parce que pas en format .jpeg")
    global index
    index = index+1
    Fenetre_choix()


In [ ]:
Fenetre_choix() #Cellule a executer pour commencer le tri d'images. S'arrete une fois que toutes les images contenues dans le dossier "Directory" ont ete classees